# Data Wrangling
<p> The first step of the process before building our model is data wrangling. The Airbnb datasets provided need to be cleaned and edited for optimal model usage. This includes performing initial feature selection, imputing missing data, examining collinearity, performing variable transformations, and further pre-processing.</p>

### Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter
%matplotlib inline

In [ ]:
# Global variables
BNB_BLUE = '#007A87'
BNB_RED = '#FF5A5F'
BNB_DARK_GRAY = '#565A5C'
BNB_LIGHT_GRAY = '#CED1CC'

### Data Exploration and Analysis
<p> We begin by examining the listings dataset below. This is the primary dataset for Airbnb listings in New York City - it contains a number of categorical and quantitative features. Through exploration and analysis we will determine which ones will be useful to our model.</p>

In [ ]:
# Read in the data 
listings = pd.read_csv('datasets/listings.csv', delimiter=',')
calendar = pd.read_csv('datasets/calendar.csv', delimiter=',', usecols=range(4))

# View feature list
print listings.columns.values

In [ ]:
# Unsuppress Output
pd.options.display.max_columns = 5

# Split into predictor and response
y = listings[['price']]

# Append price at the end of the listings table
del listings['price']
listings = listings.join(y)

# Show listings dataframe
print 'Predictor Data Shape: ', listings.shape
listings.head(n = 3)

#### Price Distribution of Listings
<p>The core listings dataset is comprised of 27,392 entries of Airbnb listing data with 51 total features including `bedrooms`, `bathrooms`, `room_type`, `review_score`, and more. We first examine the listings by price to get an initial impression of distribution.</p>

In [ ]:
# plot_hist
# Function to plot histograms
#
# Input: n (integer for subplots)
#        titles (list of titles)
#        ranges (list of values)
# Output: None
def plot_hist(n, titles, ranges):
    fig, ax = plt.subplots(n, figsize = (8, 7.5))
    for i in range(n):
        d, bins, patches = ax[i].hist(ranges[i], 50, normed = 1, color= BNB_RED, alpha = 0.85)
        ax[i].set_title(titles[i])
        ax[i].set_xlabel("Daily Listing Price in Dollars")
        ax[i].set_ylabel("Frequency")
    plt.tight_layout()
    plt.show()

In [ ]:
# Visualize price table, changing them to floats and replacing the commas with a blank
prices = listings['price'].apply(lambda s: float(s[1:].replace(',','')))

# Plot histograms of price distribution
plot_hist(3, ['Distribution of Listing Prices: All Data', 'Distribution of Listing Prices: \$0 - \$1000', 
               'Distribution of Listing Prices: \$0 - \$200'], [prices, prices[prices <= 1000], prices[prices < 250]])

<p>Above we see that prices are right skewed - most listings are below \$500 a night. Examining the price bucket \$0-\$200 (which has the vast majority of the listings) we see that listings also tend to be most frequently priced at \$100, \$150, and \$200.</p>

#### Listing Clustering from Features
<p>The right-skewness above indicates that there is likely features causing clustering in our data. Intuitively, we believe bedrooms and location can have a large impact on the price of a listing, especially in New York City. We examine this below by exploring the `bedrooms` and `neighbourhood_cleansed` features.</p>

In [ ]:
# Get frequency of bedroom number for listings
bedrooms_counts = Counter(listings.bedrooms)
tdf = pd.DataFrame.from_dict(bedrooms_counts, orient = 'index').sort_values(by = 0)
tdf = (tdf.iloc[-10:, :] / 27392) * 100

# Sort bedroom dataframe by number
tdf.sort_index(axis = 0, ascending = True, inplace = True)

# Plot percent of listings by bedroom number
ax = tdf.plot(kind = 'bar', figsize = (12, 4), color = BNB_BLUE, alpha = 0.85)
ax.set_xlabel("Number of Bedrooms")
ax.set_ylabel("Percent of Listings")
ax.set_title('Percent of Listings by Bedrooms')
ax.legend_.remove()

plt.show()

print "Percent of 1 Bedroom Listings: %{0:.2f}".format(tdf[0][1])

<p>As expected, we see that 74% of the listings are one-bedroom listings. This explains some of the right-skewness - smaller listings with less bedrooms will likely be priced cheaper than far larger ones.</p>

In [ ]:
# Get number of listings in neighborhoods
nb_counts = Counter(listings.neighbourhood_cleansed)
tdf = pd.DataFrame.from_dict(nb_counts, orient='index').sort_values(by=0)

# Plot number of listings in each neighborhood
ax = tdf.plot(kind='bar', figsize = (50,10), color = BNB_BLUE, alpha = 0.85)
ax.set_title("Neighborhoods by Number of Listings")
ax.set_xlabel("Neighborhood")
ax.set_ylabel("# of Listings")
plt.show()

print "Number of Neighborhoods:", len(nb_counts)

<p>Here we see that the majority of the neighborhoods have less than 100 listings. This reveals that geographical location will be valuable for pricing prediction. We currently have 186 neighborhoods - all of these will not be of benefit to the model so we will remove some during the data cleaning phase.</p>

In [ ]:
neighborhoods = list(set(listings['neighbourhood_cleansed'].tolist()))

p = []
for nb in neighborhoods:
    p.append(np.mean(listings[listings['neighbourhood_cleansed'] == nb]['price'].tolist()))

data = zip(p, neighborhoods)

print 'Figure 8'
print '' 

plt.figure(figsize=(186, 8))
ax = plt.bar(np.arange(len(p)), p, align='center', width = 1,  color = BNB_BLUE, alpha = 0.85)
plt.xticks(np.arange(len(p)), neighborhoods, rotation = 90)
plt.title("Avg. Price of House in Neighborhood")
plt.xlabel("Avg. Price")
plt.ylabel("Neighborhood")
plt.show()

In [ ]:
# Delete neighborhoods with less than 100 entries
for i in nb_counts.keys():
    if nb_counts[i] < 100:
        del nb_counts[i]

tdf = pd.DataFrame.from_dict(nb_counts, orient='index').sort_values(by=0)
ax = tdf.plot(kind='bar', figsize = (22,4), color = BNB_BLUE, alpha = 0.85)
ax.set_title("Neighborhoods by House # (Top 48)")
ax.set_xlabel("Neighborhood")
ax.set_ylabel("# of Listings")

plt.show()

<p>For our baseline model, we can start by removing features that we intuitively sense will not impact a listing's price. This includes 11 features:</p>

* `scrape_id`: Irrelevant to actual property data.
* `last_scraped`: All within first three days of January, irrelevant to actual property data.
* `picture_url`: Irrelevant to actual property data. 
* `host_picture_url`: Irrelevant to actual property data.
* `neighbourhood`: The `neighbourhood_cleansed` feature presents the same data in a better format.
* `state`: All listings are in the state of NY - this is useless.
* `market`: All listings are in the NY market - this is useless.
* `country`: All listings are in the USA - this is useless.
* `weekly_price`: Function of daily price - should not be a predictor.
* `monthly_price`: Function of daily price - should not be a predictor.
* `calendar_last_scraped`: All within first three days of January, irrelevant to actual property data.

In [ ]:
# Features to drop
bad_features = ['scrape_id', 'last_scraped', 'picture_url', 'host_picture_url', 'neighbourhood', 'state', 
                'market', 'country', 'weekly_price', 'monthly_price', 'calendar_last_scraped']

listings.drop(bad_features, axis=1, inplace=True)
 
print 'Predictor Data Shape: ', listings.shape

### Data Clustering

### Missing Data, Imputation, and More Feature Selection

In [ ]:
# Returns percent of missing data in column
def percent_empty(df):
    
    bools = df.isnull().tolist()
    percent_empty = float(bools.count(True)) / float(len(bools))
    
    return percent_empty

# Store emptiness for all features
emptiness = []

# Get emptiness for all features
for i in range(0, listings.shape[1]):
    emptiness.append(round(percent_empty(listings.iloc[:,i]), 2))
    
empty_dict = dict(zip(listings.columns.values.tolist(), emptiness))

# Plot emptiness graph
empty = pd.DataFrame.from_dict(empty_dict, orient = 'index').sort_values(by=0)
ax = empty.plot(kind = 'bar', color='#E35A5C', figsize = (16, 5))
ax.set_xlabel('Predictor')
ax.set_ylabel('Percent Empty/NaN')
ax.set_title('Feature Emptiness')
ax.legend_.remove()

plt.show()